In [1]:
import numpy as np
import pandas as pd
import sklearn
import pickle

import matplotlib.pyplot as plt
import cv2  # Load all models
haar = cv2.CascadeClassifier('./model/haarcascade_frontalface_default.xml')  # cascade classifier
model_svm = pickle.load(open('./model/model_svm.pickle', mode='rb'))  # machine learning model (SVM)
pca_models = pickle.load(open('./model/pca_dict.pickle', mode='rb'))  # pca dictionary
model_pca = pca_models['pca']  # PCA model
mean_face_arr = pca_models['mean_face']  # Mean Face


C:\Python Project\003 FaceRecognition-Flask-main\facerecogination\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### Create Pipeline

In [2]:
import cv2
import pickle

# Load necessary models with error handling
try:
    haar = cv2.CascadeClassifier('./model/haarcascade_frontalface_default.xml')
    if haar.empty():
        raise FileNotFoundError("Haarcascade file not found or not loaded properly.")
    
    model_svm = pickle.load(open('./model/model_svm.pickle', mode='rb'))
    pca_models = pickle.load(open('./model/pca_dict.pickle', mode='rb'))
    model_pca = pca_models['pca']
    mean_face_arr = pca_models['mean_face']
except Exception as e:
    print(f"Error loading models: {e}")
    exit(1)

# Read and preprocess the image
img = cv2.imread('./test_images/getty_test.jpg')
if img is None:
    print("Error: Image not found.")
    exit(1)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = haar.detectMultiScale(gray, 1.5, 3)

for x, y, w, h in faces:
    roi = gray[y:y + h, x:x + w]
    roi = roi / 255.0
    roi_resize = cv2.resize(roi, (100, 100), cv2.INTER_AREA if roi.shape[1] > 100 else cv2.INTER_CUBIC)
    roi_reshape = roi_resize.reshape(1, -1)
    roi_mean = roi_reshape - mean_face_arr
    eigen_image = model_pca.transform(roi_mean)

    results = model_svm.predict(eigen_image)
    prob_score = model_svm.predict_proba(eigen_image)
    prob_score_max = prob_score.max()

    # Debugging outputs
    print(f"Raw Prediction: {results}")
    print(f"Prediction Probabilities: {prob_score}")
    print(f"ROI Shape: {roi_resize.shape}")

    # Manual swap if needed
    corrected_gender = 'female' if results[0] == 'male' else 'male'
    text = f"{corrected_gender} : {prob_score_max * 100:.2f}%"

    # Assign neon colors
    color = (0, 140, 255) if corrected_gender == 'male' else (255, 20, 147)  # Neon orange for male, neon pink for female
    cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    cv2.rectangle(img, (x, y - 40), (x + w, y), color, -1)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 5)

# Display image with predictions
cv2.imshow('Face Recognition', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Raw Prediction: ['female']
Prediction Probabilities: [[0.98027212 0.01972788]]
ROI Shape: (100, 100)
Raw Prediction: ['male']
Prediction Probabilities: [[0.46095139 0.53904861]]
ROI Shape: (100, 100)
Raw Prediction: ['female']
Prediction Probabilities: [[0.97363359 0.02636641]]
ROI Shape: (100, 100)


In [2]:
results[0] = 'female' if results[0] == 'male' else 'male'
